# Notebook to generate psychometrics plot given sessions & conditions

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

import datajoint as dj
from datetime import date, timedelta
import pandas as pd
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches

from matplotlib import cm
from u19_pipeline import utility
from inspect import getmembers, isfunction
import u19_pipeline.alert_system.behavior_metrics as bm


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


### Datajoint configuration and Connection to DB

In [2]:
dj.conn()
acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
behavior = dj.create_virtual_module('behavior', 'u19_behavior')

Connecting alvaros@datajoint00.pni.princeton.edu:3306


### Find subjects w session only 2022

In [10]:
query_2021 = 'session_date between "2021-01-01" and "2021-12-31"'
query_2022 = 'session_date between "2022-01-01" and "2022-12-31"'
not_testuser     = 'subject_fullname not like "testuser%"'
not_testsubjects = 'subject_fullname not like "%_test"'
not_bad_sessions = 'is_bad_session = 0'

query_2021    = [query_2021, not_testuser, not_testsubjects, not_bad_sessions]
query_2022    = [query_2022, not_testuser, not_testsubjects, not_bad_sessions]

session_table_filtered = acquisition.Session 
for filter_key in query_2021:
    session_table_filtered = session_table_filtered & filter_key
subject_2021 = set((session_table_filtered).fetch('subject_fullname').tolist())

session_table_filtered = acquisition.Session 
for filter_key in query_2022:
    session_table_filtered = session_table_filtered & filter_key
subject_2022 = set((session_table_filtered).fetch('subject_fullname').tolist())
subject_2022

subjects_only_2022 = list(subject_2022 - subject_2021)
len(subjects_only_2022)

subjects_only_2022query = [{'subject_fullname': i} for i in subjects_only_2022]

In [11]:
session_protocol_q = 'session_protocol = "poisson_blocks.m poisson_blocks_reboot_3m.mat PoissonBlocksCondensed3m" or session_protocol = "poisson_blocks.m poisson_blocks_reboot_3m.mat PoissonBlocksCondensed3m_manuel"'
session_df = pd.DataFrame((acquisition.Session & subjects_only_2022query & session_protocol_q).fetch('KEY','session_protocol', order_by='session_start_time', as_dict=True))
session_keys = ((acquisition.Session & subjects_only_2022query & session_protocol_q).fetch('KEY', as_dict=True))
session_df

,subject_fullname,session_date,session_number,session_protocol
0,hnieh_e306,2022-01-14,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
1,hnieh_e305,2022-01-14,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
2,hnieh_e307,2022-01-14,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
3,hnieh_e308,2022-01-14,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
4,hnieh_e305,2022-01-15,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
...,...,...,...,...
2155,ms81_M026,2022-10-16,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
2156,hnieh_e3102,2022-10-16,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
2157,hnieh_e312,2022-10-16,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...
2158,hnieh_e311,2022-10-16,0,poisson_blocks.m poisson_blocks_reboot_3m.mat ...


## Subjects with session in 2022 and which protocol

In [35]:
protocol_df = session_df.groupby('session_protocol').agg({'subject_fullname': [('num_sessions', 'count'), ('num_subjects', 'nunique')]})
protocol_df.columns = protocol_df.columns.droplevel()
#protocol_df = protocol_df.reset_index()
protocol_df = protocol_df.sort_values(by='num_sessions', ascending=False)
protocol_df

,num_sessions,num_subjects
session_protocol,,
poisson_blocks.m poisson_blocks_reboot_3m.mat PoissonBlocksCondensed3m,1221,33
poisson_blocks.m poisson_blocks_reboot_3m.mat PoissonBlocksCondensed3m_manuel,939,13


### Get trials for all blocks max level & main maze

In [36]:
block_query = "main_level = level and level=11"

trials_df = pd.DataFrame((behavior.TowersBlock * behavior.TowersBlock.Trial() & block_query & session_keys).fetch('KEY', 'level', 'trial_type', 'choice', order_by='session_date', as_dict=True))
trials_df

,subject_fullname,session_date,session_number,block,trial_idx,level,trial_type,choice
0,mioffe_m51,2022-02-04,0,2,1,11,L,R
1,mioffe_m51,2022-02-04,0,2,2,11,L,R
2,mioffe_m51,2022-02-04,0,2,3,11,R,R
3,mioffe_m51,2022-02-04,0,2,4,11,L,R
4,mioffe_m51,2022-02-04,0,2,5,11,R,R
...,...,...,...,...,...,...,...,...
144344,ms81_M026,2022-10-14,0,8,35,11,R,R
144345,ms81_M026,2022-10-14,0,8,36,11,R,L
144346,ms81_M026,2022-10-14,0,8,37,11,R,R
144347,ms81_M026,2022-10-14,0,8,38,11,R,L


## Metrics by session

In [37]:
metrics_df = bm.BehaviorMetrics.get_bias_from_trial_df(trials_df, return_all_metrics=True)
metrics_df['performance'] = metrics_df['cum_correct_trials']*100 / metrics_df['cum_trials']
metrics_df

,subject_fullname,session_date,session_number,block,trial_idx,level,trial_type,choice,left_trial,right_trial,...,cum_right_trials,cum_trials,correct_trial,correct_left,correct_right,cum_correct_left_trials,cum_correct_right_trials,cum_correct_trials,bias,performance
0,hnieh_e305,2022-06-22,0,4,40,11,R,L,0,1,...,34,80,0,0,0,17,21,38,0.248082,47.500000
1,hnieh_e305,2022-06-21,0,2,109,11,R,L,0,1,...,57,109,0,0,0,31,31,62,-0.052294,56.880734
2,hnieh_e305,2022-06-20,0,4,64,11,R,L,0,1,...,62,104,0,0,0,31,27,58,-0.302611,55.769231
3,hnieh_e305,2022-06-19,0,6,1,11,L,R,1,0,...,52,107,0,0,0,29,28,57,0.011189,53.271028
4,hnieh_e305,2022-06-18,0,4,26,11,L,R,1,0,...,36,66,0,0,0,10,17,27,0.138889,40.909091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,sbolkan_a2a_478,2022-07-13,0,4,52,11,L,R,1,0,...,115,207,0,0,0,70,70,140,-0.152174,67.632850
841,sbolkan_d1_227,2022-08-12,0,2,30,11,L,L,1,0,...,13,30,1,1,0,11,9,20,0.045249,66.666667
842,sbolkan_d1_227,2022-07-14,0,2,176,11,R,L,0,1,...,81,176,0,0,0,61,49,110,-0.037167,62.500000
843,sbolkan_d1_227,2022-07-13,0,4,94,11,L,L,1,0,...,80,159,1,1,0,46,50,96,0.042722,60.377358


## Subjects with session in level 11

In [38]:
len(list(set(metrics_df['subject_fullname'].tolist())))

27

## Subjects with blocks max level > 0.7

In [42]:
metrics_df_f = metrics_df.loc[metrics_df['performance'] >= 70, :]
summary_df = metrics_df_f.groupby('subject_fullname').agg({'subject_fullname': [('num_sessions', 'count'), ('num_subjects', 'nunique')]})
summary_df.columns = summary_df.columns.droplevel()
#protocol_df = protocol_df.reset_index()
summary_df = summary_df.sort_values(by='num_sessions', ascending=False)
summary_df

,num_sessions,num_subjects
subject_fullname,,
mioffe_m60,47,1
mioffe_m59,12,1
mioffe_m51,8,1
mioffe_m62,8,1
mioffe_m58,6,1
ms81_M022,6,1
ms81_M021,5,1
hnieh_e307,4,1
mioffe_m56,4,1


In [43]:
summary_df.sum()

num_sessions    110
num_subjects     16
dtype: int64

In [44]:
metrics_df_f = metrics_df_f.sort_values(by='performance', ascending=False)
metrics_df_f

,subject_fullname,session_date,session_number,block,trial_idx,level,trial_type,choice,left_trial,right_trial,...,cum_right_trials,cum_trials,correct_trial,correct_left,correct_right,cum_correct_left_trials,cum_correct_right_trials,cum_correct_trials,bias,performance
268,mioffe_m60,2022-08-21,0,2,178,11,L,R,1,0,...,95,178,0,0,0,72,80,152,-0.025365,85.393258
239,mioffe_m60,2022-09-20,0,2,231,11,R,L,0,1,...,98,231,0,0,0,109,87,196,0.068206,84.848485
230,mioffe_m59,2022-06-23,0,2,182,11,R,L,0,1,...,83,182,0,0,0,84,70,154,-0.005111,84.615385
242,mioffe_m60,2022-09-17,0,2,179,11,L,L,1,0,...,63,179,1,1,0,90,60,150,0.176519,83.798883
260,mioffe_m60,2022-08-29,0,2,205,11,R,R,0,1,...,77,205,1,0,1,98,72,170,0.169440,82.926829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,mioffe_m56,2022-03-24,0,2,201,11,R,nil,0,1,...,109,201,0,0,0,66,75,141,-0.029318,70.149254
415,ms81_M020,2022-09-22,0,2,214,11,L,L,1,0,...,104,214,1,1,0,73,77,150,0.076748,70.093458
159,mioffe_m55,2022-03-27,0,2,224,11,R,L,0,1,...,120,224,0,0,0,76,81,157,-0.055769,70.089286
527,ms81_M021,2022-08-13,0,4,83,11,R,L,0,1,...,120,234,0,0,0,84,80,164,-0.070175,70.085470


In [30]:
query_s = dict()
query_s['subject_fullname'] = 'mioffe_m60'
query_s['session_date'] = '2022-08-21'
a = pd.DataFrame((acquisition.Session & query_s).fetch(as_dict=True))
a

,subject_fullname,session_date,session_number,session_start_time,session_end_time,session_location,task,level,set_id,stimulus_bank,stimulus_commit,session_performance,num_trials,num_trials_try,session_narrative,session_protocol,session_code_version,is_bad_session,session_comments
0,mioffe_m60,2022-08-21,0,2022-08-21 07:38:00,2022-08-21 08:40:00,165I-Rig4-T,Towers,11,1,C:/Experiments/ViRMEn/experiments/protocols/st...,2cee281cbb24f8f350990df8c6179669c886d4b9,86.0465,215,[[215.0]],,poisson_blocks.m poisson_blocks_reboot_3m.mat ...,"[[2.1, 4.0]]",0,


In [31]:
s = pd.DataFrame((behavior.TowersBlock & query_s).fetch(as_dict=True))
s

,subject_fullname,session_date,session_number,block,task,main_level,level,set_id,n_trials,first_trial,block_duration,block_start_time,reward_mil,reward_scale,easy_block,block_performance
0,mioffe_m60,2022-08-21,0,1,Towers,11,4,1,37,1,543.788,2022-08-21 07:39:00,0.1320,1,0,0.891892
1,mioffe_m60,2022-08-21,0,2,Towers,11,11,1,178,38,3131.790,2022-08-21 07:48:00,1.4144,1,0,0.853933
